In [8]:

from __future__ import print_function
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash",model_provider="google_genai")

In [9]:

def getSuggestions(weatherSummary):
    """
        To produce suggestions for user based on current weather report
    Args:
        weatherSummary : Detailed weather report
    """
    from langchain_core.prompts import ChatPromptTemplate
    prompt_template = ChatPromptTemplate([
        ("system", "You are given weather report, Tell the user how to plan their day accordingly"),
        ("user", "Make suggestions for this report {weatherSummary}")
    ])

    prompt = prompt_template.invoke({
        "weatherSummary" : weatherSummary
    })

    res = model.invoke(prompt)
    res.pretty_print()
    return res.content 


In [10]:
def getWeatherInfo(city: str) -> any:
    """
        To get the weather info in the given city

    Args:
        city : CIty in which the weather report to be fetched
    """
    
    import time
    import weatherapi
    from weatherapi.rest import ApiException
    from pprint import pprint
    import datetime

    currentDate = datetime.date.today()
    formatted_date = currentDate.strftime("%Y-%m-%d")
    # Configure API key authorization: ApiKeyAuth
    configuration = weatherapi.Configuration()
    configuration.api_key['key'] = 'e00b8e34d43b4cbea5380937250906'
    # Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
    # configuration.api_key_prefix['key'] = 'Bearer'

    # create an instance of the API class
    api_instance = weatherapi.APIsApi(weatherapi.ApiClient(configuration))
    q = city
    dt = "2025-06-11" 

    try:
        # Astronomy API
        api_response = api_instance.forecast_weather(q, dt)
        print(api_response)
        return (api_response)
    except ApiException as e:
        print("Exception when calling APIsApi->astronomy: %s\n" % e)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def makeWeatherReport(weatherReport):
    """
        Based on JSON provided by weather API
        to generate a detailed overview of the climate
    Args:
        weatherReport: JSON file containing all the weather details
    """
    prompt_template = ChatPromptTemplate([
        ("system", "You are a meteorologist. You will given a detailed JSON weather report. Summarize the important and significant points in detail"),
        ("user", "Give me weather report for {weatherJSON}")
    ]) 
    prompt = prompt_template.invoke({
        "weatherJSON" : weatherReport
    })
    response = model.invoke(prompt)
    response.pretty_print()
    return response.content

In [12]:
from langchain_core.runnables import RunnableSequence
from langchain_core.runnables import RunnableLambda
chain = RunnableLambda(getWeatherInfo) | RunnableLambda(makeWeatherReport) | RunnableLambda(getSuggestions)

In [13]:
res = chain.invoke("Madurai")
res

{'location': {'name': 'Madurai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 9.9333, 'lon': 78.1167, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1749706229, 'localtime': '2025-06-12 11:00'}, 'current': {'last_updated_epoch': 1749706200, 'last_updated': '2025-06-12 11:00', 'temp_c': 35.1, 'temp_f': 95.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.5, 'wind_kph': 13.7, 'wind_degree': 247, 'wind_dir': 'WSW', 'pressure_mb': 1004.0, 'pressure_in': 29.65, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 47, 'cloud': 50, 'feelslike_c': 43.6, 'feelslike_f': 110.4, 'windchill_c': 31.2, 'windchill_f': 88.1, 'heatindex_c': 34.5, 'heatindex_f': 94.2, 'dewpoint_c': 21.3, 'dewpoint_f': 70.3, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 9.3, 'gust_mph': 9.8, 'gust_kph': 15.7}, 'forecast': {'forecastday': [{'date': '2025-06-12', 'date_epoch': 1749686400, 'day': {'maxtemp_c': 32.4, 'maxtemp_f': 90.3, 'mintemp

'Okay, here\'s how you should plan your day in Madurai on June 12, 2025, based on the weather report:\n\n**Key Considerations:**\n\n*   **Heat & Humidity:** The "feels like" temperature is significantly higher than the actual temperature, especially in the morning.  The average humidity is also high.\n*   **High Chance of Rain:** An 85% chance of rain means you should definitely be prepared for it.\n*   **Stronger Winds:** Expect moderate to strong winds, especially during the afternoon.\n*   **Very High UV Index (Morning):** Sun protection is crucial in the morning.\n\n**Here\'s a suggested plan:**\n\n*   **Morning (Until Noon):**\n    *   **Limit Outdoor Activities:** Due to the high "feels like" temperature and very high UV index, try to minimize prolonged exposure to the sun.\n    *   **Sun Protection is a Must:** If you must be outside, wear sunscreen (SPF 30 or higher), a wide-brimmed hat, and sunglasses. Seek shade whenever possible.\n    *   **Hydration:** Drink plenty of water

In [14]:
with open("../textFiles/weather.txt", "w") as f:
    f.write(res)